In [2]:
import pandas as pd
import numpy as np
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

In [3]:
question_answer = pipeline("question-answering", model="deepset/electra-base-squad2")

In [4]:
question_answer2 = pipeline("question-answering", model="deepset/tinyroberta-squad2")

Downloading:   0%|          | 0.00/835 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/311M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/383 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [5]:
modln_name1 = "deepset/tinyroberta-squad2"
model_name2 = "deepset/electra-base-squad2"

In [6]:
df=pd.read_csv("../Dataset/final_paraphrased_df.csv")

In [7]:
df

,text
0,The Bus starts from Rwanda at 02:51 AM and goe...
1,The bus arrives at Switzerland by 8:13 AM afte...
2,The Bus starts from United Kingdom at 09:58 AM...
3,The bus arrives at United States Minor Outlyin...
4,The Bus starts from United Arab Emirates at 06...
...,...
1707,The bus leaves Hungary at 10:50 AM and arrives...
1708,The bus leaves Hungary at 10:50 AM and arrives...
1709,The bus leaves Hungary at 10:50 AM and arrives...
1710,The Bus starts from Palau at 04:15 PM and goes...


In [8]:
context = ""
for sent in df.iterrows():
    context += sent[1]['text'] + ". "

In [9]:
context

"The Bus starts from Rwanda at 02:51 AM and goes to Switzerland reaching at 8:13 AM. The bus arrives at Switzerland by 8:13 AM after departing from Rwanda at 8:13 AM. The Bus starts from United Kingdom at 09:58 AM and goes to United States Minor Outlying Islands reaching at 3:39 PM. The bus arrives at United States Minor Outlying Islands by 3:39 PM after departing from United Kingdom at 3:39 PM. The Bus starts from United Arab Emirates at 06:22 PM and goes to Bangladesh reaching at 11:55 PM. The bus arrives at Bangladesh by 11:55 PM after departing from United Arab Emirates at 11:55 PM. The Bus starts from Mexico at 09:06 AM and goes to Australia reaching at 12:02 PM. The bus arrives at Australia by 12:02 PM after departing from Mexico at 12:02 PM. The Bus starts from Hungary at 08:55 PM and goes to Angola reaching at 12:20 AM. The bus arrives at Angola by 12:20 AM after departing from Hungary at 12:20 AM. The Bus starts from Qatar at 07:58 AM and goes to Afghanistan reaching at 1:40 P

In [10]:
nlp = question_answer2
QA_input = {
    'question': 'Where does the bus from Rwanda reach to if it starts from there at 02:51 AM',
    'context': context,
}
res1 = nlp(QA_input)
print(res1)

C:\users\kamat\appdata\local\programs\python\python39\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


{'score': 0.9591224193572998, 'start': 35595, 'end': 35605, 'answer': 'Montenegro'}


In [11]:
model_deepsetelectra = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

### Using HyperParameter Tuning

In [11]:
from transformers import TrainingArguments, Trainer

In [14]:
def train(config=None):


    # set training arguments
    training_args = TrainingArguments(
        output_dir='vit-sweeps',
        report_to='wandb',  # Turn on Weights & Biases logging
        seed=42,
        batch_size = 20,
        n_epochs = 5,
        base_LM_model = "deepset/electra-base-squad2",
        max_seq_len = 120,
        learning_rate = 1e-4,
        lr_schedule = LinearWarmup,
        warmup_proportion = 0.1,
        doc_stride = 85,
        max_query_length = 60,
        load_best_model_at_end=True,
        remove_unused_columns=False,
        fp16=True
    )


    # define training loop
    trainer = Trainer(
        # model,
        model_init=model_init,
        args=training_args,
        data_collator=collate_fn,
        train_dataset=context,
        eval_dataset=datasets_processed['validation'],
        compute_metrics=compute_metrics_fn
    )


    # start training loop
    trainer.train()


In [15]:
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")
wandb.agent(sweep_id, train, count=20)


NameError: name 'wandb' is not defined

In [ ]:
trainer.train()

In [13]:
nlp = question_answer
QA_input = {
    'question': 'At what time does the bus which reaches south georgia from mexico leave at',
    'context': context,
}
res1 = nlp(QA_input)
print(res1)

{'score': 0.9991130232810974, 'start': 69522, 'end': 69530, 'answer': '02:45 PM'}


In [ ]:
QA_input = {
    'question': 'At what time does the bus which reaches south georgia from mexico leave at',
    'context': context,
}
res1 = nlp(QA_input)
print(res1)